#### El cliente ha pedido realizar un ajuste a las reglas implementadas hasta ahora, estos ajustes consisten en reemplazar algunas funciones hechas con groupBy por funciones Window, presta mucha atención y resuelve las siguientes actividades.

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de las sesiones 06, 07

In [1]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
%run utils.ipynb

In [2]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession, DataFrame, Column, Window
import pyspark.sql.functions as f
import pyspark.sql.types as t

# Creación de sesión de Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ejercicio_8") \
    .getOrCreate()

# Carga de tablas requeridas
root_path = "../resources/data/tmp/parquet/"
names_list = ["06/movies", "06/ratings", "06/tags"]
df_dict = read_tmp_df(names_list)

movies_df = df_dict["06/movies"]
ratings_df = df_dict["06/ratings"]
tags_df = df_dict["06/tags"]

movies_df.show(1, False)
ratings_df.show(1, False)
tags_df.show(1, False)

+--------+----------------+-------------------------------------------------+----+
|movie_id|title           |genres                                           |year|
+--------+----------------+-------------------------------------------------+----+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1995|
+--------+----------------+-------------------------------------------------+----+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|rating|time               |
+-------+--------+------+-------------------+
|1      |1       |4.0   |2008-11-03 11:52:19|
+-------+--------+------+-------------------+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|tag   |time               |
+-------+--------+------+-------------------+
|224183 |832     |acting|2017-06-05 08:20:27|
+-------+--------+------+-------------------+
only showing top 1 row



#### Actividad 1:
##### TO DO ->    En esta actividad hay que resolver la logica que implementaste en la "Actividad 2 de la sesión 07 - método calculate_rating_values" pero reemplazando el uso de la función groupBy por funciones Window. 
##### Tendrás que obtener el mismo resultado ya que en las validaciones se compararán ambos dataframes.
##### La firma a utilizar es la siguiente:
- ##### def calculate_rating_values_w(df: DataFrame) -> DataFrame:

In [3]:
## TU CODIGO VA EN ESTA CELDA:

def calculate_rating_values_w(df: DataFrame) -> DataFrame:
    window = Window.partitionBy(f.col("movie_id"))
    df = df.select(
        f.col("movie_id"),
        f.round(f.avg(f.col("rating")).over(window), 2).alias("avg_rating"),
        f.round(f.stddev_pop(f.col("rating")).over(window), 2).alias("stddev_rating"),
        f.count(f.col("*")).over(window).alias("count_rating"),
        f.min(f.col("time")).over(window).alias("min_time_rating")
    ).distinct()
    return df # modificar codigo interno

In [4]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
calculate_rating_values_w(ratings_df).show(2)
"""
Ejemplo de salida esperada:
+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|  100062|      3.63|         0.83|          64|2014-03-11 21:23:33|
|  100070|      3.54|         0.89|          13|2013-01-24 11:24:50|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows
"""

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|  100062|      3.63|         0.83|          64|2014-03-11 21:23:33|
|  100070|      3.54|         0.89|          13|2013-01-24 11:24:50|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows



'\nEjemplo de salida esperada:\n+--------+----------+-------------+------------+-------------------+\n|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|\n+--------+----------+-------------+------------+-------------------+\n|  100062|      3.63|         0.83|          64|2014-03-11 21:23:33|\n|  100070|      3.54|         0.89|          13|2013-01-24 11:24:50|\n+--------+----------+-------------+------------+-------------------+\nonly showing top 2 rows\n'

In [5]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
rating_values_df = calculate_rating_values_w(ratings_df)

assert type(rating_values_df) == DataFrame
assert len(rating_values_df.columns) == 5
assert rating_values_df.count() == 83239

data = rating_values_df \
    .filter(f.col("movie_id") == "296") \
    .collect()[0]

assert data["movie_id"] == "296"
assert data["avg_rating"] == 4.19
assert data["stddev_rating"] == 0.95
assert data["count_rating"] == 108756
assert data["min_time_rating"] == datetime.datetime(1996, 2, 29, 10, 48, 44)

In [6]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(rating_values_df, "09/ratings")]

write_tmp_df(dfs)

#### Actividad 2:
##### TO DO ->    En la siguiente celda se realiza la validación de que el contenido de la función desarrollada calculate_rating_values_w sea identico al contenido del dataframe correspondiente del ejercicio "Actividad 2 de la sesión 07 - método calculate_rating_values".
##### Para realizar la validación nos apoyamos cualquiera de las siguientes funciones de la clase DataFrame:
- subtract (en caso de spark-scala la función es except)-> https://spark.apache.org/docs/3.2.1/api/python/reference/api/pyspark.sql.DataFrame.subtract.html
- exceptAll -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.exceptAll.html#pyspark.sql.DataFrame.exceptAll
##### Estas funciones las vas a utilizar mucho en tus futuros proyectos, basicamente al aplicar la acción "count" al dataframe retornado de aplicar esta función (subtract o exceptAll) entre dos dataframes identicos el valor resultante debería ser cero.
##### Asumimos que en este caso el resultado de subtract debería ser cero. En caso de que el valor resultante sea distinto de cero coloca las transformaciones necesarias para localizar qué registros tienen la diferencia y si es posible hacer una corrección, realizala en la "Actividad 1" (método calculate_rating_values_w) o simplemente justifica  la diferencia de ambos dataframes.

In [7]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

root_path = "../resources/data/tmp/parquet/"
names_list = ["07/ratings", "09/ratings"]
rates_dict = read_tmp_df(names_list)
ratings_values_07_df = rates_dict["07/ratings"]
ratings_values_09_df = rates_dict["09/ratings"]

diff_01 = ratings_values_09_df.exceptAll(ratings_values_07_df)
print("Conteo de diferencias del dataframe actual (ratings_values_09_df) vs dataframe de Actividad 2 sesion 07: ", diff_01.count())
diff_01.show()
diff_02 = ratings_values_07_df.exceptAll(ratings_values_09_df)
print("Conteo de diferencias del dataframe de Actividad 2 sesion 07 vs dataframe actual (ratings_values_09_df): ", diff_02.count())
diff_02.show()

print("Justificación\n La desviación estándar es la que difiere, esto es por temas de redondeo, quiero suponer que varía la manera de hacer el cálculo de la desviación estándar ya que se hace un proceso de shuffle con el groupBy")

Conteo de diferencias del dataframe actual (ratings_values_09_df) vs dataframe de Actividad 2 sesion 07:  2
+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|   82356|      3.38|         0.63|          16|2010-12-09 13:07:28|
|  194474|      3.63|         0.88|          16|2019-02-20 08:41:51|
+--------+----------+-------------+------------+-------------------+

Conteo de diferencias del dataframe de Actividad 2 sesion 07 vs dataframe actual (ratings_values_09_df):  2
+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|  194474|      3.63|         0.87|          16|2019-02-20 08:41:51|
|   82356|      3.38|         0.62|          16|2010-12-09 13:07:28|
+--------+----------+---

In [8]:
## Aqui van tus transformación para revisar y/o justificar diferencias

diff_01 # Transformaciones a diff_01 (Revisión)
diff_02 # Transformaciones a diff_02 (Revisión)
print("Justificación\n La desviación estándar es la que difiere debido a la forma en la que se efectúa el cálculo, para este caso, con dos decimales se puede ver esta diferencia, si se emokean más decimales, sería complicado ver estas diferencias")

Justificación
 La desviación estándar es la que difiere debido a la forma en la que se efectúa el cálculo, para este caso, con dos decimales se puede ver esta diferencia, si se emokean más decimales, sería complicado ver estas diferencias


#### Actividad 3:
##### TO DO ->    En esta actividad hay que resolver la logica que implementaste en la "Actividad 3 de la sesión 07 - métodos get_act_3_df1 y get_act_3_df2" pero reemplazando el uso de la función groupBy por funciones Window. 
##### Tendrás que obtener el mismo resultado ya que en las validaciones se compararán ambos dataframes.
##### La firma a utilizar es la siguiente:
- ##### Primer tabla (utilizando función concat_ws o concat): def get_act_3_df1_w(df: DataFrame) -> DataFrame:
- ##### Segunda tabla (utilizando función struct): def get_act_3_df2_w(df: DataFrame) -> DataFrame:

In [9]:
## TU CODIGO VA EN ESTA CELDA, PUEDES GENERAR MÉTODOS O VARIABLES NUEVAS SI ASI LO REQUIERES

def get_act_3_df1_w(df: DataFrame):
    window_1 = Window.partitionBy(f.col("movie_id"))

    df = df.select(
        *difference(tags_df.columns,["tag"]),
        f.upper(f.col("tag")).alias("tag")
        )
    window_2 = Window.partitionBy(f.col("movie_id"),f.col("tag"))
    df = df.select(
        f.col("movie_id"),
        f.col("tag"),
        f.count("tag").over(window_2).alias("count"),
        f.min(f.col("time")).over(window_1).alias("min")
    ).distinct()
    df = df.select(
        f.col("movie_id"),
        f.col("min"),
        f.concat_ws(":", f.col("tag"), f.col("count").cast(t.StringType())).alias("tag_count")
    )
    df = df.select(
        f.col("movie_id"),
        f.collect_set(f.col("tag_count")).over(window_1).alias("tag_count"),
        f.min(f.col("min")).over(window_1).alias("min_time_tag")
    ).distinct()
    
    df = df.select(
        f.col("movie_id"),
        f.sort_array(f.col("tag_count")).alias("tag_count"),
        f.col("min_time_tag")
    )
    return df  # ... transformaciones a tags_df

def get_act_3_df2_w(df: DataFrame):
    window_1 = Window.partitionBy(f.col("movie_id"))
    df = df.select(
        *difference(tags_df.columns,["tag"]),
        f.upper(f.col("tag")).alias("tag")
        )

    window_2 = Window.partitionBy(f.col("movie_id"),f.col("tag"))

    df = df.select(
        f.col("movie_id"),
        f.col("tag"),
        f.count("tag").over(window_2).alias("count"),
        f.min(f.col("time")).over(window_1).alias("min")
    ).distinct()

    df = df.select(
        f.col("movie_id"),
        f.collect_set(f.struct(f.col("tag"), f.col("count"))).over(window_1).alias("tag_count"),
        f.min(f.col("min")).over(window_1).alias("min_time_tag")
    ).distinct()

    df = df.select(
        f.col("movie_id"),
        f.sort_array(f.col("tag_count")).alias("tag_count"),
        f.col("min_time_tag")
    )
    
    return df

act_3_df1 = get_act_3_df1_w(tags_df)
act_3_df2 = get_act_3_df2_w(tags_df)

In [10]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
act_3_df1.show(2, False)
act_3_df2.show(2, False)
"""
Ejemplo de salida esperada:
+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_rating    |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[PRESS-GANGED : 1, WORLD WAR II : 1, WAR : 1, FATE : 1]           |2018-05-26 16:40:54|
|100070  |[GOOD HUMOUR : 1, STRUGGLING CAREER : 1, COMEDY : 1, COMEDIAN : 2]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_rating    |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{PRESS-GANGED, 1}, {WORLD WAR II, 1}, {WAR, 1}, {FATE, 1}]           |2018-05-26 16:40:54|
|100070  |[{GOOD HUMOUR, 1}, {STRUGGLING CAREER, 1}, {COMEDY, 1}, {COMEDIAN, 2}]|2017-05-19 17:17:36|
+--------+----------------------------------------------------------------------+-------------------+
only showing top 2 rows
"""

+--------+----------------------------------------------------------+-------------------+
|movie_id|tag_count                                                 |min_time_tag       |
+--------+----------------------------------------------------------+-------------------+
|100062  |[FATE:1, PRESS-GANGED:1, WAR:1, WORLD WAR II:1]           |2018-05-26 17:40:54|
|100070  |[COMEDIAN:2, COMEDY:1, GOOD HUMOUR:1, STRUGGLING CAREER:1]|2017-05-19 18:17:36|
+--------+----------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_tag       |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{FATE, 1}, {PRESS-GANGED, 1}, {WAR, 1}, {WORLD WAR II, 1}]           |2018-05-26 17:40:54|
|100070  |[{COMEDIAN, 2}, {

'\nEjemplo de salida esperada:\n+--------+------------------------------------------------------------------+-------------------+\n|movie_id|tag_count                                                         |min_time_rating    |\n+--------+------------------------------------------------------------------+-------------------+\n|100062  |[PRESS-GANGED : 1, WORLD WAR II : 1, WAR : 1, FATE : 1]           |2018-05-26 16:40:54|\n|100070  |[GOOD HUMOUR : 1, STRUGGLING CAREER : 1, COMEDY : 1, COMEDIAN : 2]|2017-05-19 17:17:36|\n+--------+------------------------------------------------------------------+-------------------+\nonly showing top 2 rows\n\n+--------+----------------------------------------------------------------------+-------------------+\n|movie_id|tag_count                                                             |min_time_rating    |\n+--------+----------------------------------------------------------------------+-------------------+\n|100062  |[{PRESS-GANGED, 1}, {WORLD W

In [11]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert len(act_3_df1.columns) == 3
assert "movie_id" in act_3_df1.columns
assert "tag_count" in act_3_df1.columns
assert "min_time_tag" in act_3_df1.columns
assert schema_to_ddl(act_3_df1.select("movie_id", "tag_count", "min_time_tag")) == 'movie_id STRING,tag_count ARRAY<STRING> NOT NULL,min_time_tag TIMESTAMP'

assert len(act_3_df2.columns) == 3
assert "movie_id" in act_3_df2.columns
assert "tag_count" in act_3_df2.columns
assert "min_time_tag" in act_3_df2.columns
assert schema_to_ddl(act_3_df2.select("movie_id", "tag_count", "min_time_tag")) == 'movie_id STRING,tag_count ARRAY<STRUCT<tag: STRING, count: BIGINT>> NOT NULL,min_time_tag TIMESTAMP'

#### Actividad 4:
##### TO DO ->    En la siguiente celda se realiza la validación de que el dataframe retornado por la función desarrollada get_act_3_df2_w sea identico al contenido del dataframe correspondiente del ejercicio "Actividad 3 de la sesión 07 - método get_act_3_df2".
##### Para realizar la validación nos apoyamos de cualquiera de las siguientes funciones de la clase DataFrame:
- subtract (en caso de spark-scala la función es except)-> https://spark.apache.org/docs/3.2.1/api/python/reference/api/pyspark.sql.DataFrame.subtract.html
- exceptAll -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.exceptAll.html#pyspark.sql.DataFrame.exceptAll
##### Probablemente existan diferencias en la comparación de ambos dataframes, investiga por qué ocurren dichas diferencias y haz los ajustes necesarios en tu actividad 3 o en la celda donde se aplica la función subtract para que la diferencia mostrada sea cero.
##### Probablemente requieras utilizar la siguiente función:
- sort_array -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sort_array.html#pyspark.sql.functions.sort_array

In [12]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(act_3_df1, "09/tags_p1"),
       (act_3_df2, "09/tags_p2")]

write_tmp_df(dfs)

In [13]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
root_path = "../resources/data/tmp/parquet/"
names_list = ["07/tags_p1", "07/tags_p2", "09/tags_p1", "09/tags_p2"]
rates_dict = read_tmp_df(names_list)
tags_values_07_p1_df = rates_dict["07/tags_p1"]
tags_values_07_p2_df = rates_dict["07/tags_p2"]
tags_values_09_p1_df = rates_dict["09/tags_p1"]
tags_values_09_p2_df = rates_dict["09/tags_p2"]

In [14]:
# Puedes modificar esta celda si asi lo requieres, también puedes modificar tu desarrollo en la actividad 3 (método get_act_3_df2_w). 
# Pudes aplicar algunas transformaciones a los dataframes tags_values_07_p2_df y tags_values_09_p2_df únicamente para reordenar datos.
# El resultado de las operaciones count debe ser cero ya que ambos dataframes deberían contener la misma información.

#Recuerda que si el conteo da cero entonces ambos dataframes son identicos.
diff_01 = tags_values_07_p2_df.exceptAll(tags_values_09_p2_df)
print("Conteo de diferencias del dataframe actual (tags_values_09_p2_df) vs dataframe de Actividad 2 sesion 07: ", diff_01.count())
diff_02 = tags_values_09_p2_df.exceptAll(tags_values_07_p2_df)
print("Conteo de diferencias del dataframe de Actividad 2 sesion 07 vs dataframe actual (tags_values_09_p2_df): ", diff_02.count())

Conteo de diferencias del dataframe actual (tags_values_09_p2_df) vs dataframe de Actividad 2 sesion 07:  0
Conteo de diferencias del dataframe de Actividad 2 sesion 07 vs dataframe actual (tags_values_09_p2_df):  0


In [15]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
assert diff_01.count() == 0
assert diff_02.count() == 0

In [16]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
last_movies_df = movies_df \
    .join(tags_values_09_p2_df, ["movie_id"], "left") \
    .join(ratings_values_09_df, ["movie_id"], "left")

last_movies_df.show()

+--------+--------------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
|movie_id|               title|              genres|year|           tag_count|       min_time_tag|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+--------------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
|       1|    Toy Story (1995)|[Adventure, Anima...|1995|[{1990S, 1}, {200...|2006-01-12 19:19:35|      3.89|         0.93|       76813|1996-01-28 18:00:00|
|       2|      Jumanji (1995)|[Adventure, Child...|1995|[{1860S, 1}, {196...|2006-01-13 14:14:30|      3.28|         0.96|       30209|1996-01-28 18:00:00|
|       3|Grumpier Old Men ...|   [Comedy, Romance]|1995|[{ANN MARGARET, 1...|2006-01-16 00:28:45|      3.17|         1.02|       15820|1996-01-29 17:54:45|
|       4|Waiting to Exhale...|[Comedy, Drama, R...|1995|[

#### Actividad 5:
##### TO DO ->    La última actividad a realizar es la siguiente:
- ##### El cliente nos ha solicitado generar una tabla donde se muestre el top de peliculas ranqueadas por genero, para realizar esto se realizan los siguientes pasos:
    - ##### Las peliculas que entran a la parte del ranqueo deben cumplir con las siguientes condiciones:
        -  count_rating > 1000
        -  avg_rating >= 4.2
        -  stddev_rating < 2
    - #####  Descomponer la columna "genres" en una columna llamada "genre"
    - ##### Agregar la columna "top" donde se asignará el valor de la función "rank" de Spark tomando las siguientes caracteristicas:
        - particionar por: "genre"
        - order por: avg_rating DESC, stddev_rating ASC, count_rating DESC
##### El esquema resultante deberá ser:
* |-- title: string
* |-- year: integer
* |-- genre: string
* |-- top: integer se)

In [20]:
## TU CODIGO VA EN ESTA CELDA:

last_movies_df = movies_df \
    .join(tags_values_09_p2_df, ["movie_id"], "left") \
    .join(ratings_values_09_df, ["movie_id"], "left")

last_movies_df = last_movies_df \
    .select(
        *last_movies_df.columns
    ) \
    .where((f.col("count_rating") > 1000) & (f.col("avg_rating") >= 4.2) & (f.col("stddev_rating") < 2))

last_movies_df = last_movies_df \
    .select(
        *difference(last_movies_df.columns, ["genres"]),
        f.explode(f.col("genres")).alias("genre")
    )

window = Window.partitionBy(f.col("genre")).orderBy(f.col("avg_rating").desc(), f.col("stddev_rating").asc(),
                                                    f.col("count_rating").desc())

top_movies_df = last_movies_df \
    .select(
        f.col("title"),
        f.col("year"),
        f.col("genre"),
        f.rank().over(window).alias("top")
    )


In [21]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

top_movies_df.show(5)
"""
Ejemplo de salida esperada (hay que mantener mismo orden de columnas):
+--------------------+----+---------+---+
|               title|year|    genre|top|
+--------------------+----+---------+---+
|Band of Brothers ...|2001|   Action|  1|
|Seven Samurai (Sh...|1954|   Action|  2|
|   Fight Club (1999)|1999|   Action|  3|
|Over the Garden W...|2013|Adventure|  1|
|Seven Samurai (Sh...|1954|Adventure|  2|
+--------------------+----+---------+---+
only showing top 5 rows
"""

+--------------------+----+---------+---+
|               title|year|    genre|top|
+--------------------+----+---------+---+
|Band of Brothers ...|2001|   Action|  1|
|Seven Samurai (Sh...|1954|   Action|  2|
|   Fight Club (1999)|1999|   Action|  3|
|Over the Garden W...|2013|Adventure|  1|
|Seven Samurai (Sh...|1954|Adventure|  2|
+--------------------+----+---------+---+
only showing top 5 rows



'\nEjemplo de salida esperada (hay que mantener mismo orden de columnas):\n+--------------------+----+---------+---+\n|               title|year|    genre|top|\n+--------------------+----+---------+---+\n|Band of Brothers ...|2001|   Action|  1|\n|Seven Samurai (Sh...|1954|   Action|  2|\n|   Fight Club (1999)|1999|   Action|  3|\n|Over the Garden W...|2013|Adventure|  1|\n|Seven Samurai (Sh...|1954|Adventure|  2|\n+--------------------+----+---------+---+\nonly showing top 5 rows\n'

In [22]:
assert top_movies_df.orderBy(f.col("genre").asc(), f.col("top").asc()).collect() == top_movies